# Principal Component Analysis
* `dim2` -- PCA Minka's MLE
* `dim3` -- PCA required Explained Variance

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# add path
import sys; import os; sys.path.append(os.path.realpath("../"))
import pandas as pd

In [3]:
# demo datasets
from datasets.demo1 import X_train, Y_train, fold_ids, X_valid, Y_valid, meta as meta_data
#meta_data

In [4]:
# transformer implementations
typ = 'dim3'

if typ is 'dim2':
    from verto.dim2 import trans, meta
elif typ is 'dim3':
    from verto.dim3 import trans, meta
    trans.set_params(**{'required_ev': 0.5})  # slowly increase from 0.1 towards 100%

meta

{'id': 'dim3',
 'name': 'PCA req EV',
 'description': 'Number of components is determined by a required Explained Variance threshold',
 'keywords': ['dimensionality reduction',
  'principal component anlysis',
  'StandardScaler',
  'PCA',
  'Explained Variance']}

## Transform

In [5]:
%%time
trans.fit(X_train)

CPU times: user 4.51 ms, sys: 2.54 ms, total: 7.06 ms
Wall time: 5.28 ms


PcaEV(prefix='pcaev', required_ev=0.5)

In [6]:
%%time
X_new = trans.transform(X_train)

CPU times: user 1.42 ms, sys: 1.72 ms, total: 3.14 ms
Wall time: 6.68 ms


In [7]:
df_new = pd.DataFrame(data=X_new, columns=trans.feature_names_)

## Evaluate
- check if the PCA components are "good" predictors
- eyeball the p-values of the logistic regression coefficients

In [8]:
df_new.head()

,pcaev_0,pcaev_1
0,-3.083984,-2.158203
1,-2.183594,-0.617676
2,2.050781,2.328125
3,-2.808594,-0.950684
4,-1.913086,-1.072266


In [9]:
import statsmodels.api as sm
#lr = sm.Logit(Y_train, sm.add_constant(X_new)).fit()
lr = sm.Logit(Y_train, sm.add_constant(X_new)).fit_regularized(method='l1', alpha=.5)
print(lr.summary())

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.14263565385478444
            Iterations: 30
            Function evaluations: 30
            Gradient evaluations: 30
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  398
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            2
Date:                Mon, 25 Mar 2019   Pseudo R-squ.:                  0.7906
Time:                        09:48:42   Log-Likelihood:                -55.106
converged:                       True   LL-Null:                       -263.17
                                        LLR p-value:                 4.335e-91
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------